In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Descarga de recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Función para limpiar y procesar el texto
def preprocess_text(text):
    if pd.isna(text):
        text = ''  # Reemplazar NaN por una cadena vacía
    tokens = word_tokenize(text.lower())  # Tokenización y conversión a minúsculas
    tokens = [word for word in tokens if word.isalnum()]  # Eliminar puntuación
    tokens = [word for word in tokens if word not in stopwords.words('spanish')]  # Eliminar stopwords en español
    return ' '.join(tokens)

# Función para comparar las recomendaciones y determinar si son similares
def are_similar(text1, text2):
    preprocessed_text1 = preprocess_text(text1)
    preprocessed_text2 = preprocess_text(text2)
    
    # Utilizar TF-IDF para representar los textos como vectores
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([preprocessed_text1, preprocessed_text2])
    
    # Calcular la similitud coseno entre los vectores
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    
    # Si la similitud es alta, considerar que las recomendaciones son similares
    return similarity > 0.7  # Puedes ajustar este umbral según tus necesidades

# Leer las bases de datos desde archivos Excel
database1 = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Recomendaciones históricas ArCo.xlsx')
database2 = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Consolidado 2024.xlsx')

# Reemplazar NaN por cadenas vacías en las columnas relevantes
database1['Recomendación de mejora (Max. 100 palabras)'] = database1['Recomendación de mejora (Max. 100 palabras)'].fillna('')
database2['Recomendación de mejora (Max. 100 palabras)'] = database2['Recomendación de mejora (Max. 100 palabras)'].fillna('')

# Lista para almacenar las recomendaciones diferentes y sus datos adicionales
different_recommendations = []

# Comparar cada recomendación de database1 con todas las recomendaciones de database2
for index1, row1 in database1.iterrows():
    is_different = True
    for index2, row2 in database2.iterrows():
        if are_similar(row1['Recomendación de mejora (Max. 100 palabras)'], row2['Recomendación de mejora (Max. 100 palabras)']):
            is_different = False
            break  # Romper el bucle interior si se encuentra una recomendación similar
    if is_different:
        different_recommendations.append(row1)

# Crear un DataFrame con las recomendaciones diferentes y sus datos adicionales
columns_to_keep = [
    'Código recomendación', 'Tipo Recomendación', 'Nivel ArCo', 'Código(s) de instrumento(s)',
    '$ PGN', '$ SGR', '$ otras fuentes', 'Sector', 'Entidad', 'Nombre de la oferta',
    'Recomendación de mejora (Max. 100 palabras)', 'Justificación de la recomendación (Max. 100 palabras)', 
    'Entidad / Área', 'Comité Técnico SNCI', 'Formulador', 'Contacto', 
    'No. Contacto', 'Implementada', 'Justificación'
]
different_recommendations_df = pd.DataFrame(different_recommendations, columns=columns_to_keep)

# Exportar el DataFrame a un archivo Excel
different_recommendations_df.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/diferentes_recomendaciones.xlsx', index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Recomendaciones únicas y agrupaciones


In [14]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el modelo de spaCy en español
nlp = spacy.load('es_core_news_md')

# Función para obtener el vector de una recomendación
def get_vector(text):
    return nlp(text).vector

# Cargar el archivo Excel
df = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Recomendaciones históricas ArCo.xlsx')

# Asegurarse de que solo se procesen filas con texto en la columna de recomendaciones
df = df.dropna(subset=['Recomendación de mejora (Max. 100 palabras)'])
df['Recomendación de mejora (Max. 100 palabras)'] = df['Recomendación de mejora (Max. 100 palabras)'].astype(str)

# Aplicar la función a la columna de recomendaciones para obtener sus vectores
df['vector'] = df['Recomendación de mejora (Max. 100 palabras)'].apply(get_vector)

# Calcular la matriz de similitud coseno
vectors = np.vstack(df['vector'].values)
similarity_matrix = cosine_similarity(vectors)

# Umbral de similitud para considerar dos recomendaciones como duplicadas
threshold = 0.8

# Crear una lista para guardar las agrupaciones
groups = []

# Identificar duplicados y obtener el top 3 de recomendaciones similares
to_drop = set()
for i in range(len(similarity_matrix)):
    similarities = []
    for j in range(len(similarity_matrix)):
        if i != j and similarity_matrix[i, j] > threshold:
            similarities.append((j, similarity_matrix[i, j]))
    # Ordenar las recomendaciones similares por similitud y seleccionar el top 3
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]
    similar_indices = [sim[0] for sim in similarities]
    if similar_indices:
        group = [i] + similar_indices
        groups.append(group)
        to_drop.update(similar_indices)

# Crear un DataFrame para las agrupaciones
df_groups = pd.DataFrame(columns=['Recomendación Principal', 'Recomendaciones Similares'])

for group in groups:
    main_rec = df.iloc[group[0]]['Recomendación de mejora (Max. 100 palabras)']
    similar_recs = [df.iloc[idx]['Recomendación de mejora (Max. 100 palabras)'] for idx in group[1:]]
    new_row = pd.DataFrame({
        'Recomendación Principal': [main_rec],
        'Recomendaciones Similares': [", ".join(similar_recs)]
    })
    df_groups = pd.concat([df_groups, new_row], ignore_index=True)

# Crear un nuevo DataFrame sin duplicados
df_unique = df.drop(df.index[list(to_drop)])

# Eliminar la columna de vectores antes de guardar
df_unique = df_unique.drop(columns=['vector'])

# Guardar el resultado en nuevos archivos Excel
df_unique.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_unicas.xlsx', index=False)
df_groups.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/agrupaciones_similares.xlsx', index=False)

print("Proceso completado. El archivo con recomendaciones únicas se ha guardado como 'recomendaciones_unicas.xlsx'.")
print("El archivo con agrupaciones de recomendaciones similares se ha guardado como 'agrupaciones_similares.xlsx'.")


c:\Users\sebas\anaconda3\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_md' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Proceso completado. El archivo con recomendaciones únicas se ha guardado como 'recomendaciones_unicas.xlsx'.
El archivo con agrupaciones de recomendaciones similares se ha guardado como 'agrupaciones_similares.xlsx'.


# Pruebas

In [15]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el modelo de spaCy en español
nlp = spacy.load('es_core_news_md')

# Función para obtener el vector de una recomendación
def get_vector(text):
    return nlp(text).vector

# Cargar el archivo Excel
df = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Recomendaciones históricas ArCo.xlsx')

# Asegurarse de que solo se procesen filas con texto en la columna de recomendaciones
df = df.dropna(subset=['Recomendación de mejora (Max. 100 palabras)'])
df['Recomendación de mejora (Max. 100 palabras)'] = df['Recomendación de mejora (Max. 100 palabras)'].astype(str)

# Aplicar la función a la columna de recomendaciones para obtener sus vectores
df['vector'] = df['Recomendación de mejora (Max. 100 palabras)'].apply(get_vector)

# Calcular la matriz de similitud coseno
vectors = np.vstack(df['vector'].values)
similarity_matrix = cosine_similarity(vectors)

# Umbral de similitud para considerar dos recomendaciones como duplicadas
threshold = 0.8

# Crear una lista para guardar las agrupaciones
groups = []
main_indices = []

# Identificar duplicados y obtener el top 3 de recomendaciones similares
for i in range(len(similarity_matrix)):
    similarities = []
    for j in range(len(similarity_matrix)):
        if i != j and similarity_matrix[i, j] > threshold:
            similarities.append((j, similarity_matrix[i, j]))
    # Ordenar las recomendaciones similares por similitud y seleccionar el top 3
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]
    similar_indices = [sim[0] for sim in similarities]
    if similar_indices:
        group = [i] + similar_indices
        groups.append(group)
        main_indices.append(i)

# Crear un DataFrame para las agrupaciones
df_groups = pd.DataFrame(columns=['Recomendación Principal', 'Recomendaciones Similares'])

for group in groups:
    main_rec = df.iloc[group[0]]['Recomendación de mejora (Max. 100 palabras)']
    similar_recs = [df.iloc[idx]['Recomendación de mejora (Max. 100 palabras)'] for idx in group[1:]]
    new_row = pd.DataFrame({
        'Recomendación Principal': [main_rec],
        'Recomendaciones Similares': [", ".join(similar_recs)]
    })
    df_groups = pd.concat([df_groups, new_row], ignore_index=True)

# Crear un nuevo DataFrame con las recomendaciones principales
df_unique = df.iloc[main_indices].reset_index(drop=True)

# Eliminar la columna de vectores antes de guardar
df_unique = df_unique.drop(columns=['vector'])

# Guardar el resultado en nuevos archivos Excel
df_unique.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_unicas.xlsx', index=False)
df_groups.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/agrupaciones_similares.xlsx', index=False)

print("Proceso completado. El archivo con recomendaciones únicas se ha guardado como 'recomendaciones_unicas.xlsx'.")
print("El archivo con agrupaciones de recomendaciones similares se ha guardado como 'agrupaciones_similares.xlsx'.")


c:\Users\sebas\anaconda3\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_md' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Proceso completado. El archivo con recomendaciones únicas se ha guardado como 'recomendaciones_unicas.xlsx'.
El archivo con agrupaciones de recomendaciones similares se ha guardado como 'agrupaciones_similares.xlsx'.


In [18]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el modelo de spaCy en español
nlp = spacy.load('es_core_news_md')

# Función para obtener el vector de una recomendación
def get_vector(text):
    return nlp(text).vector

# Cargar el archivo Excel
df = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Recomendaciones históricas ArCo.xlsx')

# Asegurarse de que solo se procesen filas con texto en la columna de recomendaciones
df = df.dropna(subset=['Recomendación de mejora (Max. 100 palabras)'])
df['Recomendación de mejora (Max. 100 palabras)'] = df['Recomendación de mejora (Max. 100 palabras)'].astype(str)

# Aplicar la función a la columna de recomendaciones para obtener sus vectores
df['vector'] = df['Recomendación de mejora (Max. 100 palabras)'].apply(get_vector)

# Calcular la matriz de similitud coseno
vectors = np.vstack(df['vector'].values)
similarity_matrix = cosine_similarity(vectors)

# Umbral de similitud para considerar dos recomendaciones como duplicadas
threshold = 1

# Crear una lista para guardar las agrupaciones
groups = []
main_indices = []

# Identificar duplicados y obtener el top 3 de recomendaciones similares
for i in range(len(similarity_matrix)):
    similarities = []
    for j in range(len(similarity_matrix)):
        if i != j and similarity_matrix[i, j] > threshold:
            similarities.append((j, similarity_matrix[i, j]))
    # Ordenar las recomendaciones similares por similitud y seleccionar el top 3
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]
    similar_indices = [sim[0] for sim in similarities]
    if similar_indices:
        group = [i] + similar_indices
        groups.append(group)
        main_indices.append(i)

# Eliminar duplicados en main_indices
main_indices = list(set(main_indices))

# Crear un DataFrame para las agrupaciones
df_groups = pd.DataFrame(columns=['Recomendación Principal', 'Recomendaciones Similares'])

for group in groups:
    main_rec = df.iloc[group[0]]['Recomendación de mejora (Max. 100 palabras)']
    similar_recs = [df.iloc[idx]['Recomendación de mejora (Max. 100 palabras)'] for idx in group[1:]]
    new_row = pd.DataFrame({
        'Recomendación Principal': [main_rec],
        'Recomendaciones Similares': [", ".join(similar_recs)]
    })
    df_groups = pd.concat([df_groups, new_row], ignore_index=True)

# Crear un nuevo DataFrame con las recomendaciones principales
df_unique = df.iloc[main_indices].reset_index(drop=True)

# Eliminar la columna de vectores antes de guardar
df_unique = df_unique.drop(columns=['vector'])

# Contar las recomendaciones únicas y las recomendaciones principales
unique_recommendations_count = df_unique['Recomendación de mejora (Max. 100 palabras)'].nunique()
main_recommendations_count = df_groups['Recomendación Principal'].nunique()

# Imprimir los resultados
print(f"Recuento distintivo de recomendaciones únicas: {unique_recommendations_count}")
print(f"Recuento distintivo de recomendaciones principales: {main_recommendations_count}")

# Guardar el resultado en nuevos archivos Excel
df_unique.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_unicas.xlsx', index=False)
df_groups.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/agrupaciones_similares.xlsx', index=False)

print("Proceso completado. El archivo con recomendaciones únicas se ha guardado como 'recomendaciones_unicas.xlsx'.")
print("El archivo con agrupaciones de recomendaciones similares se ha guardado como 'agrupaciones_similares.xlsx'.")



c:\Users\sebas\anaconda3\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_md' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Recuento distintivo de recomendaciones únicas: 29
Recuento distintivo de recomendaciones principales: 29
Proceso completado. El archivo con recomendaciones únicas se ha guardado como 'recomendaciones_unicas.xlsx'.
El archivo con agrupaciones de recomendaciones similares se ha guardado como 'agrupaciones_similares.xlsx'.


# Duplicados

Duplicados Recomendaciones Históricas


In [23]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Recomendaciones históricas ArCo.xlsx')

# Asegurarse de que solo se procesen filas con texto en la columna de recomendaciones
df = df.dropna(subset=['Recomendación de mejora (Max. 100 palabras)'])

# Crear una nueva columna concatenada con las columnas relevantes
df['concatenated'] = df[['Tipo Recomendación', 'Nivel ArCo', 'Sector', 'Entidad', 'Nombre de la oferta', 'Recomendación de mejora (Max. 100 palabras)']].astype(str).agg(' '.join, axis=1)

# Identificar duplicados: keep=False marca todas las ocurrencias como duplicadas
df['Validación de duplicados'] = df.duplicated('concatenated', keep=False).map({True: 'Se repite', False: 'No se repite'})

# Crear DataFrame para las filas duplicadas
df_duplicates = df[df['Validación de duplicados'] == 'Se repite']

# Crear DataFrame para las filas únicas
df_unique = df[df['Validación de duplicados'] == 'No se repite']

# Eliminar la columna concatenada antes de guardar
df = df.drop(columns=['concatenated'])
df_duplicates = df_duplicates.drop(columns=['concatenated'])
df_unique = df_unique.drop(columns=['concatenated'])

# Guardar los resultados en nuevos archivos Excel
df_duplicates.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_duplicadas.xlsx', index=False)
df_unique.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_historicas_sin_duplicados.xlsx', index=False)

print("Proceso completado. Los archivos se han guardado como 'recomendaciones_duplicadas.xlsx' y 'recomendaciones_unicas.xlsx'.")


Proceso completado. Los archivos se han guardado como 'recomendaciones_duplicadas.xlsx' y 'recomendaciones_unicas.xlsx'.


Duplicados Consolidado de recomendaciones


In [2]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/20240529 – Balance Implementación de Articulaciones ArCo.xlsx')

# Asegurarse de que solo se procesen filas con texto en la columna de recomendaciones
df = df.dropna(subset=['Recomendación de mejora (Max. 100 palabras)'])

# Crear una nueva columna concatenada con las columnas relevantes
df['concatenated'] = df[['Nivel ArCo', 'Nombre de la oferta', 'Código(s) de instrumento(s)', 'Recomendación de mejora (Max. 100 palabras)', 'Justificación de la recomendación (Max. 100 palabras)']].astype(str).agg(' '.join, axis=1)

# Identificar duplicados: keep=False marca todas las ocurrencias como duplicadas
df['Validación de duplicados'] = df.duplicated('concatenated', keep=False).map({True: 'Se repite', False: 'No se repite'})

# Crear DataFrame para las filas duplicadas
df_duplicates = df[df['Validación de duplicados'] == 'Se repite']

# Crear DataFrame para las filas únicas
df_unique = df[df['Validación de duplicados'] == 'No se repite']

# Eliminar la columna concatenada antes de guardar
df = df.drop(columns=['concatenated'])
df_duplicates = df_duplicates.drop(columns=['concatenated'])
df_unique = df_unique.drop(columns=['concatenated'])

# Guardar los resultados en nuevos archivos Excel
df_duplicates.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_duplicadas_final.xlsx', index=False)
df_unique.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_historicas_sin_duplicados_consolidado.xlsx', index=False)

print("Proceso completado. Los archivos se han guardado como 'recomendaciones_duplicadas.xlsx' y 'recomendaciones_unicas.xlsx'.")


Proceso completado. Los archivos se han guardado como 'recomendaciones_duplicadas.xlsx' y 'recomendaciones_unicas.xlsx'.


# Punto 5

In [2]:
pip install fuzzywuzzy

  Obtaining dependency information for fuzzywuzzy from https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from fuzzywuzzy import fuzz

# Cargar las bases de datos
consolidado_path = 'C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Consolidado 2024.xlsx'
instrumentos_path = 'C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/20220725 -  Instrumentos articulados ArCo 2021 y 2022_RevCPGPP para python_01.xlsx'

# Leer los datos
consolidado_df = pd.read_excel(consolidado_path)
instrumentos_df = pd.read_excel(instrumentos_path)



In [6]:
#consolidado_df.head()
instrumentos_df.head()

,N°,Año de implementación,Código recomendación,Tipo Recomendación,Nivel ArCo,Código(s) de instrumento(s),$ PGN,$ SGR,$ otras fuentes,Sector,Entidad,Nombre de la oferta,Recomendación de mejora (Max. 100 palabras),Justificación de la recomendación (Max. 100 palabras),Entidad / Área (formulador de la recomendación),Comité Técnico SNCI sobre el cual se puede tratar el tema,Formulador de la recomendación,Contacto del formulador,Evidencia de articulación
0,1,2021,2020048,2. Directriz,III. Definir roles,6004,0.0,0,3000.0,06 - Agropecuario,AGROSAVIA,SERVICIOS Y PRODUCTOS DE LABORATORIOS,Mejoras en el diseño o características de los ...,El instrumento debe alinearse con la red de la...,DNP - DDRS,NaN,Natalia Acosta,17007,Se cuenta con un convenio marco firmado para a...
1,2,2021,2020048,2. Directriz,III. Definir roles,6051,35602.0,0,0.0,06 - Agropecuario,ICA,"Servicio de análisis y diagnóstico sanitario, ...",NaN,NaN,NaN,NaN,NaN,NaN,Se cuenta con un convenio marco firmado para a...
2,3,2021,2020048,2. Directriz,III. Definir roles,6055,275.0,0,0.0,06 - Agropecuario,ICA,Servicio de registro a laboratorios externos,NaN,NaN,NaN,NaN,NaN,NaN,Se cuenta con un convenio marco firmado para a...
3,4,2021,2020061,2. Directriz,II. Fusionar,8001,4251.0,0,1859.0,08 - Comunicaciones,Ministerio de Tecnologías de la información y ...,Programación para niños y niñas,Fusionar / Empaquetar instrumentos dentro del ...,"En el proceso de revisión, se evidencia que am...",DNP - DDD,"Ciencia, Tecnología e Innovación",Laura Lorena Flórez Jiménez,3815000 Ext. 12757,El proyecto de code.org no continúa en la vige...
4,5,2021,2020062,2. Directriz,II. Fusionar,8016,7895.0,0,0.0,08 - Comunicaciones,Ministerio de Tecnologías de la Información y ...,Integración de Trámites y Servicios de las ent...,Fusionar / Empaquetar instrumentos dentro del ...,"En el proceso de revisión, se evidencia que am...",DNP - DDD,"Ciencia, Tecnología e Innovación",Laura Lorena Flórez Jiménez,3815000 Ext. 12757,"Desde MinTIC, consideramos que no es convenien..."


In [8]:
# Asegurarse de que las columnas de interés existen en ambos DataFrames
# Ajusta los nombres de las columnas según tus archivos reales

# Crear las nuevas columnas en el DataFrame consolidado
consolidado_df['Instrumentos Articulados – Reporte Presidencia'] = ''
consolidado_df['Año de implementación'] = ''

# Definir las columnas a comparar
columns_to_compare = ['Tipo Recomendación', 'Nivel ArCo', 'Nombre de la oferta', 'Recomendación de mejora (Max. 100 palabras)', 'Justificación de la recomendación (Max. 100 palabras)']

# Función para comparar dos filas
def compare_rows(row1, row2, threshold=90):
    for col in columns_to_compare:
        if pd.isna(row1[col]) or pd.isna(row2[col]):
            return False
        if fuzz.ratio(str(row1[col]), str(row2[col])) < threshold:
            return False
    return True

# Iterar sobre cada fila en el DataFrame de instrumentos
for idx, instrumento_row in instrumentos_df.iterrows():
    # Buscar coincidencias en el DataFrame consolidado
    for reco_idx, reco_row in consolidado_df.iterrows():
        if compare_rows(instrumento_row, reco_row):
            consolidado_df.at[reco_idx, 'Instrumentos Articulados – Reporte Presidencia'] = instrumento_row['Código(s) de instrumento(s)']
            consolidado_df.at[reco_idx, 'Año de implementación'] = instrumento_row['Año de implementación']

# Guardar el DataFrame actualizado a un nuevo archivo de Excel
consolidado_df.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Consolidado 2024 Actualizado.xlsx', index=False)


# Actualización código de instrumentos - Base Articulados vs Base ArCo 2024


In [18]:
import pandas as pd
from fuzzywuzzy import process, fuzz
#from fuzzywuzzy import fuzz

# Cargar las bases de datos
df_2024 = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Originales/Base ArCo 2024.xlsx')
df_2021_2022 = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Originales/20220725 -  Instrumentos articulados ArCo 2021 y 2022_RevCPGPP para python_01.xlsx')

# Crear una función para encontrar el código más similar
def find_best_match(nombre, choices, cutoff=60):
    match, score = process.extractOne(nombre, choices, scorer=fuzz.ratio)
    if score >= cutoff:
        return match
    return None

# Lista de nombres y códigos de la base de 2024
choices = df_2024['Nombre del Instrumento'].tolist()
codigos = dict(zip(df_2024['Nombre del Instrumento'], df_2024['Código']))

# Encontrar los mejores matches
df_2021_2022['Nombre del Instrumento 2024'] = df_2021_2022['Nombre de la oferta'].apply(find_best_match, choices=choices, cutoff=80)

# Asignar los códigos correspondientes
df_2021_2022['Código 2024'] = df_2021_2022['Nombre del Instrumento 2024'].map(codigos)

# Eliminar la columna temporal de nombres
df_2021_2022.drop(columns=['Nombre del Instrumento 2024'], inplace=True)

# Guardar la base resultante
df_2021_2022.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Originales/instrumentos_comparados_01.xlsx', index=False)



# Actualización código de instrumentos - Base Consolidada vs Base ArCo 2024


In [20]:
import pandas as pd
from fuzzywuzzy import process, fuzz
#from fuzzywuzzy import fuzz

# Cargar las bases de datos
df_2024 = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Originales/Base ArCo 2024.xlsx')
df_consolidada= pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Originales/Base Consolidada.xlsx')

# Asegurar que todas las entradas en 'Nombre de la oferta' son de tipo string
df_consolidada['Nombre de la oferta'] = df_consolidada['Nombre de la oferta'].astype(str)

# Crear una función para encontrar el código más similar
def find_best_match(nombre, choices, cutoff=60):
    match, score = process.extractOne(nombre, choices, scorer=fuzz.ratio)
    if score >= cutoff:
        return match
    return None

# Lista de nombres y códigos de la base de 2024
choices = df_2024['Nombre del Instrumento'].tolist()
codigos = dict(zip(df_2024['Nombre del Instrumento'], df_2024['Código']))

# Encontrar los mejores matches
df_consolidada['Nombre del Instrumento 2024'] = df_consolidada['Nombre de la oferta'].apply(find_best_match, choices=choices, cutoff=80)

# Asignar los códigos correspondientes
df_consolidada['Código 2024'] = df_consolidada['Nombre del Instrumento 2024'].map(codigos)

# Eliminar la columna temporal de nombres
df_consolidada.drop(columns=['Nombre del Instrumento 2024'], inplace=True)

# Guardar la base resultante
df_consolidada.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/Originales/instrumentos_comparados_final.xlsx', index=False)



# Duplicados con procesamiento de lenguaje natural

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Read the Excel spreadsheet into a pandas DataFrame
df = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/20240529 – Balance Implementación de Articulaciones ArCo.xlsx')
df.head()

c:\Users\sebas\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,Código recomendación,Tipo Recomendación,Nivel ArCo,Código(s) de instrumento(s),$ PGN,$ SGR,$ otras fuentes,Sector,Entidad,Nombre de la oferta,...,Entidad / Área,Comité Técnico SNCI,Formulador,Contacto,No. Contacto,Implementada,Justificación,Instrumentos Articulados – Reporte Presidencia,Año de implementación,Código 2024
0,2021046,4. Mensajes,Otra,333,1169.966677,0,0,Sector Ambiente y Desarrollo Sostenible,INSTITUTO AMAZÓNICO DE INVESTIGACIONES CIENTÍF...,Servicio de modelamiento para la conservación ...,...,DADS/GBSE,Sostenibilidad,Rosanna Ovalle,rovalle@dnp.gov.co,NaN,Sin retroalimentación,NaN,NaN,NaN,2440.0
1,2021047,4. Mensajes,Otra,334,2228.634584,0,0,Sector Ambiente y Desarrollo Sostenible,INSTITUTO AMAZÓNICO DE INVESTIGACIONES CIENTÍF...,Documentos de estudios técnicos para la gestió...,...,DADS/GBSE,Sostenibilidad,Rosanna Ovalle,rovalle@dnp.gov.co,NaN,Sin retroalimentación,NaN,NaN,NaN,2450.0
2,2021048,4. Mensajes,Otra,335,40.000000,0,0,Sector Ambiente y Desarrollo Sostenible,AUTORIDAD NACIONAL DE LICENCIAS AMBIENTALES ANLA,Retos de Innovación 2022,...,DADS/Cambio Climático,Sostenibilidad,Angélica Becerra,ABEcerra@dnp.gov.co,NaN,No,Los retos de innovación de la ANLA surgen como...,NaN,NaN,NaN
3,2021049,4. Mensajes,Otra,411,45.000000,0,0,Sector Ambiente y Desarrollo Sostenible,AUTORIDAD NACIONAL DE LICENCIAS AMBIENTALES ANLA,Curso Virtual de Licenciamiento Ambiental,...,DADS/Cambio Climático\nDADS/Sectorial,Sostenibilidad,Angélica Becerra\nLuis Elkin Guzmán Ramos,abecerra@dnp.gov.co\nlguzman@dnp.gov.co,NaN,No,"""El curso virtual de Licenciamiento Ambiental ...",NaN,NaN,NaN
4,2021050,4. Mensajes,Otra,483,0.000000,0,1460,Sector Ambiente y Desarrollo Sostenible,AUTORIDAD NACIONAL DE LICENCIAS AMBIENTALES ANLA,Seguimiento Documental Espacial - SDE,...,DADS/Cambio Climático,Sostenibilidad,Angélica Becerra,abecerra@dnp.gov.co,NaN,No,1. Se está validando la estrategia para la con...,NaN,NaN,NaN


In [6]:
# Read the Excel spreadsheet into a pandas DataFrame
df = pd.read_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/20240529 – Balance Implementación de Articulaciones ArCo.xlsx')


# Definir las columnas relevantes
columns_to_compare = ['Código(s) de instrumento(s)', 'Nombre de la oferta','Nivel ArCo', 'Recomendación de mejora (Max. 100 palabras)']

# Asegurarse de que todos los valores en 'Código(s) de instrumento(s)' sean strings
df['Código(s) de instrumento(s)'] = df['Código(s) de instrumento(s)'].astype(str)


# Crear una columna combinada con todas las columnas relevantes para facilitar la comparación
df['combined'] = df[columns_to_compare].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Definir un umbral de similitud
threshold = 100

# Inicializar la columna 'Estado' con 'No se repite'
df['Estado'] = 'No se repite'

# Encontrar y marcar duplicados
for i, row in df.iterrows():
    for j, other_row in df.iloc[i+1:].iterrows():
        similarity = fuzz.ratio(row['combined'], other_row['combined'])
        if similarity >= threshold:
            df.at[i, 'Estado'] = 'Se repite'
            df.at[j, 'Estado'] = 'Se repite'

# Eliminar la columna combinada antes de exportar
df.drop(columns=['combined'], inplace=True)

# Exportar el DataFrame modificado a un nuevo archivo de Excel
df.to_excel('C:/Backup - Archivos de trabajo/DNP/Tareas/2024/Mayo/2024 05 27 - Algoritmo para validar recomendaciones históricas/Insumo/recomendaciones_marcadas.xlsx', index=False)
